In [76]:
from lonboard import viz, PathLayer
from lonboard.colormap import apply_categorical_cmap
import pandas as pd
import geopandas as gpd
import numpy as np

In [2]:
# From inspecting pbf files
layers = ['points', 'lines', 'multilinestrings', 'multipolygons']

gdf_list = []

# Iterate through the layers and read each one
for layer in layers:
    try:
        # Read the layer from the PBF file
        gdf = gpd.read_file("../data/raw/output/europe_toilets.pbf", engine="pyogrio", layer=layer)
        
        # Add a new column to indicate the layer
        gdf['layer'] = layer
        
        # Append the GeoDataFrame to the list
        gdf_list.append(gdf)

        # Optionally print the first few rows of the GeoDataFrame
        print(f"Layer: {layer}, Number of features: {len(gdf)}")

        # Export the layer as a separate .geojson, to later add as tile layers in tippecanoe

        if len(gdf) > 0:
            export = gdf[["osm_id", "geometry"]]
            export.to_file(f"../data/raw/geojsons/{layer}.geojson", driver="GeoJSON")
        else:
            print(f"Nothing to write to {layer}.geojson. Skipping...")
    except Exception as e:
        print(f"Failed to read layer {layer}: {e}")

# Concatenate all GeoDataFrames in the list into a single GeoDataFrame
combined_gdf = gpd.GeoDataFrame(pd.concat(gdf_list, ignore_index=True))

combined_gdf_min = combined_gdf[["osm_id", "geometry"]]

Layer: points, Number of features: 136058
Layer: lines, Number of features: 94
Layer: multilinestrings, Number of features: 0
Nothing to write to multilinestrings.geojson. Skipping...
Layer: multipolygons, Number of features: 45465


In [3]:
viz(combined_gdf_min)

Map(basemap_style=<CartoBasemap.DarkMatter: 'https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json'…

In [4]:
# Help for the engine: https://github.com/geopandas/geopandas/discussions/3015

water = gpd.read_file("../data/raw/output/europe_water.pbf", engine="pyogrio", layer = "points")
toilets = gpd.read_file("../data/raw/output/europe_toilets.pbf", engine="pyogrio", layer="points")
benches = gpd.read_file("../data/raw/output/europe_benches.pbf", engine="pyogrio", layer="points")

In [5]:
print(len(water), "drinking water POIs,", len(toilets), "toilet POIs, and", len(benches), "bench/picnic table POIs")

248904 drinking water POIs, 136058 toilet POIs, and 2382296 bench/picnic table POIs


In [6]:
water = water[["osm_id", "name", "geometry", "other_tags"]]
toilets = toilets[["osm_id", "name", "geometry", "other_tags"]]

In [7]:
viz([toilets, water])

Map(basemap_style=<CartoBasemap.DarkMatter: 'https://basemaps.cartocdn.com/gl/dark-matter-gl-style/style.json'…

In [8]:
water_min = water[["osm_id", "geometry"]]
# water_min.to_file("../data/water.geojson", driver="GeoJSON")

In [9]:
toilets_min = toilets[["osm_id", "geometry"]]
# toilets_min.to_file("../data/toilets.geojson", driver="GeoJSON")

---

## SAC Scale

In [27]:
# Function to extract sac_scale from other_tags
def extract_sac_scale(other_tags):
    if pd.isna(other_tags) or 'sac_scale' not in other_tags:
        return None
    import re
    match = re.search(r'"sac_scale"=>\"([^\"]+)\"', other_tags)
    return match.group(1) if match else None

In [118]:
# Function to extract sac_scale from other_tags
def extract_sac_scale(other_tags):
    if pd.isna(other_tags) or 'sac_scale' not in other_tags:
        return None
    import re
    match = re.search(r'"sac_scale"=>\"([^\"]+)\"', other_tags)
    return match.group(1) if match else None

# Load hiking paths data and save to a variable
sac_gdf = gpd.read_file("../data/raw/output/sac.pbf", engine="pyogrio", layer="lines")
if 'other_tags' in sac_gdf.columns:
    # Add sac_scale column
    sac_gdf['sac_scale'] = sac_gdf['other_tags'].apply(extract_sac_scale)
    
    # Show the distribution of sac_scale values
    print("SAC Scale Distribution:")
    print(sac_gdf['sac_scale'].value_counts(dropna=True))

# Now sac_gdf contains your data with the sac_scale column


SAC Scale Distribution:
sac_scale
hiking                                         27734
mountain_hiking                                18645
demanding_mountain_hiking                       3218
alpine_hiking                                   1415
demanding_alpine_hiking                          500
difficult_alpine_hiking                          216
service                                            3
hiking;mountain_hiking                             2
mountain_way                                       1
mountain_hiking;hiking                             1
T4 - difficult, exposed, steep alpine trail        1
3                                                  1
Name: count, dtype: int64


In [119]:
sac_gdf = sac_gdf[["osm_id", "name", "geometry", "sac_scale"]]

In [126]:
# 1) Define your six allowed scales → codes 0–5
code_map = {
    'strolling':                    0,
    'hiking':                       1,
    'mountain_hiking':              2,
    'demanding_mountain_hiking':    3,
    'alpine_hiking':                4,
    'demanding_alpine_hiking':      5,
    'difficult_alpine_hiking':      6,
}

# 2) Filter to only those rows, so we never see any unmapped values
mask = sac_gdf['sac_scale'].isin(code_map)
sac_clean = sac_gdf[mask].copy()

# 3) Create the 0–5 codes
sac_clean['sac_code'] = sac_clean['sac_scale'].map(code_map)
codes = sac_clean['sac_code'].to_numpy().astype(int)

# 4) Define your six colors in [R,G,B] for codes 0→5:
colors = [
    [255, 255,   0],  # 0 → yellow
    [255, 255,   0],  # 1 → yellow
    [255, 165,   0],  # 2 → orange
    [255,   0,   0],  # 3 → red
    [  0,   0, 255],  # 4 → blue
    [128,   0, 128],  # 5 → purple
    [255,   0, 255],  # 6 → magenta
]

# 5) Build the Lonboard layer, passing your filtered GeoDataFrame via `data=`
layer = PathLayer.from_geopandas(
    gdf=sac_clean,
    get_color=apply_categorical_cmap(codes, colors),
    width_min_pixels = 2
)

# Now add `layer` to your Deck and render!

In [137]:
Voyager = 'https://basemaps.cartocdn.com/gl/voyager-gl-style/style.json'

In [138]:
m = Map(layer, basemap_style=Voyager)
m

Map(basemap_style='https://basemaps.cartocdn.com/gl/voyager-gl-style/style.json', custom_attribution='', layer…